<a href="https://colab.research.google.com/github/matan034/KneeFemurPrediction/blob/main/KneeProject_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Predict



In [ ]:
# install required packages
!pip install numpy-stl
!pip install trimesh

In [ ]:
import os,glob
from google.colab import drive, files
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import nibabel as nib
from stl import mesh
import trimesh
from datetime import datetime


In [ ]:

IMAGE_HEIGHT =  256
IMAGE_WIDTH = 256
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
SCALE_FACTOR = 0.3
drive.mount('/content/drive')

TRANSFER_LEARNING = True  # use attention unet with transfer learning
CLASSIFIER = True         # use attention unet with a classifer
EXTRA_LAYER = True        # use attention unet with extra layer
USE_NII_FROM_DRIVE = True
json_model_path = '/content/drive/MyDrive/KneeProject/Best results/Attention with everything/all_Attention-UNET_(256_256)_epochs:150_learning:0.0001_loss:Tver_batch:64.json'
h5_model_path = '/content/drive/MyDrive/KneeProject/Best results/Attention with everything/all_Attention-UNET_(256_256)_epochs:150_learning:0.0001_loss:Tver_batch:64.h5'
if USE_NII_FROM_DRIVE:
  filename = '/content/drive/MyDrive/KneeProject/volume_data/'
else:
  uploaded = files.upload()
  for file in uploaded:
    filename = file

In [ ]:
# Load model architecture from JSON file
with open(json_model_path, 'r') as json_file:
    loaded_model_json = json_file.read()
    loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# Load weights into loaded model
loaded_model.load_weights(h5_model_path)

model = loaded_model


print('Model loaded from disk.')

In [ ]:
VOLUME_SLICE_X=True   # use X axis in prediction
VOLUME_SLICE_Y=True   # use Y axis in prediction
VOLUME_SLICE_Z=True   # use Z axis in prediction
HOUNSFIELD_MIN = -1000
HOUNSFIELD_MAX = 2000
HOUNSFIELD_RANGE = HOUNSFIELD_MAX - HOUNSFIELD_MIN

def scaleImg(img, height, width):
    return cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_LINEAR)

# Normalize image
def normalizeImageIntensityRange(img):
    img[img < HOUNSFIELD_MIN] = HOUNSFIELD_MIN
    img[img > HOUNSFIELD_MAX] = HOUNSFIELD_MAX
    return (img - HOUNSFIELD_MIN) / HOUNSFIELD_RANGE

def predictVolume(inImg, toBin=True):
    (xMax, yMax, zMax) = inImg.shape

    outImgX = np.zeros((xMax, yMax, zMax))
    outImgY = np.zeros((xMax, yMax, zMax))
    outImgZ = np.zeros((xMax, yMax, zMax))

    cnt = 0.0
    if VOLUME_SLICE_X:
        cnt += 1.0
        for i in range(xMax):
            img = inImg[i,:,:]
            img = scaleImg(img, IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            if TRANSFER_LEARNING or CLASSIFIER:
              img = np.repeat(img, 3, axis=3)
            tmp = model.predict(img)[0,:,:,0]
            outImgX[i,:,:] = scaleImg(tmp, yMax, zMax)
    if VOLUME_SLICE_Y:
        cnt += 1.0
        for i in range(yMax):
            img = scaleImg(inImg[:,i,:], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            if TRANSFER_LEARNING or CLASSIFIER:
              img = np.repeat(img, 3, axis=3)
            tmp = model.predict(img)[0,:,:,0]
            outImgY[:,i,:] = scaleImg(tmp, xMax, zMax)
    if VOLUME_SLICE_Z:
        cnt += 1.0
        for i in range(zMax):
            img = scaleImg(inImg[:,:,i], IMAGE_HEIGHT, IMAGE_WIDTH)[np.newaxis,:,:,np.newaxis]
            if TRANSFER_LEARNING or CLASSIFIER:
              img = np.repeat(img, 3, axis=3)
            tmp = model.predict(img)[0,:,:,0]
            outImgZ[:,:,i] = scaleImg(tmp, xMax, yMax)

    outImg = (outImgX + outImgY + outImgZ)/cnt
    if(toBin):
        outImg[outImg>0.5] = 1.0
        outImg[outImg<=0.5] = 0.0
    return outImg

In [ ]:
imgTargetNii = nib.load(filename)
# Get the image data as a 3D array
img_data = imgTargetNii.get_fdata()
imgTarget = normalizeImageIntensityRange(img_data)
predImg = predictVolume(imgTarget)

In [ ]:
vertices, faces, _, _ = measure.marching_cubes(predImg)

affine_matrix = imgTargetNii.affine


# Add a homogeneous coordinate (1) to each vertex to make the matrix multiplication possible
homogeneous_vertices = np.hstack((vertices, np.ones((len(vertices), 1))))

# Apply the affine transformation
vertices = np.dot(homogeneous_vertices, affine_matrix.T)[:, :3]  # Only keep the x, y, z coordinates
reflection_matrix = np.array([[-1, 0, 0],
                              [0, -1, 0],
                              [0, 0, 1]])

# Apply the reflection to the mesh vertices
vertices = np.dot(vertices, reflection_matrix)
knee_mesh=trimesh.Trimesh(vertices,faces)
knee_mesh=trimesh.smoothing.filter_laplacian(knee_mesh,0.5,20)

timestamp = datetime.now()
mesh_path = f'knee-segmented{timestamp}.stl'
knee_mesh.export(mesh_path)

files.download(mesh_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>